# Implementing binary search

You've seen how to implement a recursive version of the binary search algorithm. Since you avoided copying lists you ended up with a running time of $O(\log n)$. I've included the final implementation here, for your reference:

In [1]:
def binary_search(x, lst):
    def recurse(start, end):
        # Inv: start <= end and x might be in lst[start:end].
        if start == end: #start = 0 (finish search)
            return False
        i = start + (end - start) // 2
        if lst[i] == x:
            return True
        # Inv: start < end
        elif lst[i] < x:
            return recurse(i + 1, end)
        else:
            return recurse(start, i)
        
    return recurse(0, len(lst))

binary_search(1000, [1,2,3,4,5])

False

In this exercise you will implement the binary search algorithm in $O(\log n)$ time, but without recursion. We'll then compare the performance of the two implementations.

To get started with your implementation remember that the basic steps of the algorithm of the same. However, in the recursive implementation you *implicitly* loop until `start == end` and in the non-recursive implementation you will have to loop *explicitly*.

In [15]:
def binary_search_nonrec(x, lst):
    start = 0
    end = len(lst)
    while start != end:
        i = start + (end - start) // 2
        if lst[i] == x:
            return True
        elif lst[i] < x:
            start, end = i + 1, end
        else:
            start, end = start, i

    return False

binary_search_nonrec(1000, [1, 2, 3, 4, 5])

False

# Testing with an oracle

To test our implementation we'll use a neat trick which in software testing is known as an *oracle* (more specifically a *consistency oracle*). The basic idea is to compare the outputs of implementation A with output from implementation B, where B is a simpler (or well-tested) implementation. That is, B is the oracle.

Since we're already pretty sure that the recursive implementation is correct we can use it as our oracle. If we wanted to make sure that the recursive implementation was correct, we could use the simple linear search algorithm as the oracle.

Anyway, we'll write a simple test using our oracle:

In [16]:
my_lst = [0, 7, 8, 13, 42, 78, 91]
for i in range(101):
    binary_search_res = binary_search(i, my_lst)
    binary_search_nonrec_res = binary_search_nonrec(i, my_lst)
    if binary_search_res != binary_search_nonrec_res:
        print('binary_search_nonrec =', binary_search_res)
        print('binary_search =', binary_search_nonrec_res)

Testing with an oracle is especially useful when you need to test on complex data where computing the result by hand is time-consuming and error-prone.

# Which implementation is faster?

Our two search algorithms are both $O(\log n)$ so asymptotically they are equally fast. However, let's see how they perform in the real world:

In [17]:
%timeit binary_search_nonrec(8, my_lst)
%timeit binary_search(8, my_lst)

The slowest run took 17.33 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 578 ns per loop
1000000 loops, best of 3: 1.06 µs per loop


You'll probably see (unless you have a very funky computer) that the non-recursive implementation is faster than the recursive one. That's the constant we were talking about in the lecture. On my computer, the recursive implementation is almost 2x slower than the non-recursive implementation and thus the recursive implementation must have some overhead somewhere. Let's take a look at that!

When Python code is executed the code is first translated to bytecode. Bytecode is essentially a list of instructions which perform very specific actions. In Python we can easily see the bytecode using the `dis` function:

In [20]:
import dis
dis.dis(binary_search)
print("\n")
dis.dis(binary_search_nonrec)

  2           0 LOAD_CLOSURE             0 (lst)
              2 LOAD_CLOSURE             1 (recurse)
              4 LOAD_CLOSURE             2 (x)
              6 BUILD_TUPLE              3
              8 LOAD_CONST               1 (<code object recurse at 0x7f68e0176c90, file "<ipython-input-1-5b36ec19eda8>", line 2>)
             10 LOAD_CONST               2 ('binary_search.<locals>.recurse')
             12 MAKE_FUNCTION            8
             14 STORE_DEREF              1 (recurse)

 15          16 LOAD_DEREF               1 (recurse)
             18 LOAD_CONST               3 (0)
             20 LOAD_GLOBAL              0 (len)
             22 LOAD_DEREF               0 (lst)
             24 CALL_FUNCTION            1
             26 CALL_FUNCTION            2
             28 RETURN_VALUE


  2           0 LOAD_CONST               1 (0)
              2 STORE_FAST               2 (start)

  3           4 LOAD_GLOBAL              0 (len)
              6 LOAD_FAST             

The first column specifies that line number (one line of Python code may translate to many instructions), the second column is the position of the instruction in the byte code, the third column is the human-readable name of the instruction, and the fourth column is an index that specifies an argument to the instruction. The `dis()` function is nice and shows what the argument corresponds to in the last column.

If you're curious you can find a list of all Python bytecode instructions [here](https://docs.python.org/3/library/dis.html#python-bytecode-instructions).

So what's going on here? We won't go into detail, but we can see that every time `binary_search()` is called, Python must load a *code object* corresponding to the inner function (`recurse()`), make that into a function object, and then call the function. Those are pretty expensive instructions that are not really related to binary search.

To avoid building the function object for `recurse()` every time we call `binary_search()`, let's try to pull it out of the function:

In [21]:
def recurse(x, lst, start, end):
    # Inv: start <= end and x might be in lst[start:end].
    if start == end:
        return False
    i = start + (end - start) // 2
    if lst[i] == x:
        return True
    # Inv: start < end
    elif lst[i] < x:
        return recurse(x, lst, i + 1, end)
    else:
        return recurse(x, lst, start, i)

def binary_search_1(x, lst):
    return recurse(x, lst, 0, len(lst))

In [22]:
dis.dis(binary_search_1)

 15           0 LOAD_GLOBAL              0 (recurse)
              2 LOAD_FAST                0 (x)
              4 LOAD_FAST                1 (lst)
              6 LOAD_CONST               1 (0)
              8 LOAD_GLOBAL              1 (len)
             10 LOAD_FAST                1 (lst)
             12 CALL_FUNCTION            1
             14 CALL_FUNCTION            4
             16 RETURN_VALUE


There's now a lot fewer instructions! The function object for `recurse()` is built once (when the function is defined), so we simply have to load the `recurse()` function object, set up the arguments and then call it. So is it any faster now?

In [23]:
%timeit binary_search_1(8, my_lst)

The slowest run took 13.53 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 830 ns per loop


On my computer that's actually faster, but it still doesn't beat the non-recursive function. Can we make it even faster? Well, the `binary_search_1()` function doesn't really do much work, it just calls `recurse`. Maybe we don't need to wrap `recurse()` at all:

In [24]:
def binary_search_2(x, lst, start=0, end=None):
    if end is None:
        end = len(lst)
    # Inv: start <= end and x might be in lst[start:end].
    if start == end:
        return False
    i = start + (end - start) // 2
    if lst[i] == x:
        return True
    # Inv: start < end
    elif lst[i] < x:
        return recurse(x, lst, i + 1, end)
    else:
        return recurse(x, lst, start, i)

In [26]:
%timeit binary_search_2(8, my_lst)

The slowest run took 10.77 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 851 ns per loop


In this version we completely got rid of the wrapping function and we get another improvement in speed. However, on my computer this implementation is still a bit slower than the non-recursive implementation.

This teaches us something: function calls are slow. We just removed the call to `recurse()` and got a quite significant speed improvement, even though we had to add an `if`-statement to our function. So function calls must be really slow.

This ends our optimization journey (or "exploration of the constant"). The code for the recursive and non-recursive implementations are now so similar that the only difference is that the recursive implementation is recursive and thus has to use the slow function calls. The only way to make it faster is to write it iteratively, which you did in the beginning of this exercise.

In [ ]:
def binary_search_nonrec(x, lst):
    start = 0
    end = len(lst)
    while start != end:
        i = start + (end - start) // 2
        if lst[i] == x:
            return True
        elif lst[i] < x:
            start, end = i + 1, end
        else:
            start, end = start, i

    return False

def binary_search_2(x, lst, start=0, end=None):
    if end is None:
        end = len(lst)
    # Inv: start <= end and x might be in lst[start:end].
    if start == end:
        return False
    i = start + (end - start) // 2
    if lst[i] == x:
        return True
    # Inv: start < end
    elif lst[i] < x:
        return recurse(x, lst, i + 1, end)
    else:
        return recurse(x, lst, start, i)